In [8]:
import polars as pl

In [9]:
(pitching :=
 pl.read_csv('./data/lahman_1871-2024_csv/Pitching.csv')
)
 

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zuverge01""",1958,1,"""BAL""","""AL""",2,2,45,0,0,0,7,207,74,26,4,17,22,0.28,3.39,3,2,6,0,294,23,29,null,null,null
"""zuverge01""",1959,1,"""BAL""","""AL""",0,1,6,0,0,0,0,39,15,6,1,6,1,0.3,4.15,0,1,0,0,55,1,7,null,null,null
"""zychto01""",2015,1,"""SEA""","""AL""",0,0,13,1,0,0,0,55,17,5,1,3,24,0.239,2.45,0,1,2,0,76,4,6,0,0,0


In [10]:
import polars as pl
pitching_ERA = (pitching
    .filter(pl.col("yearID") >= 1946)
    .group_by("yearID", "lgID")
    .agg(
        total_runs_allowed = pl.sum("R"),
        total_innings_pitched = pl.sum("IPouts"),
    )
    .with_columns(
        # add the ERA
        league_wide_ERA = pl.col("total_runs_allowed") / pl.col("total_innings_pitched") * 27,
        
        # add the DH_rule column
        DH_rule = pl.when((pl.col("lgID") == "AL") & (pl.col("yearID") >= 1973))
            .then(pl.lit("DH"))
            .when((pl.col("lgID") == "NL") & (pl.col("yearID") >= 2022))
            .then(pl.lit("DH"))
            .otherwise(pl.lit("No DH"))
    )
    .drop("total_innings_pitched", "total_runs_allowed")
)

In [7]:
pitching_ERA.write_csv("data/pitching_ERA.csv")

In [14]:
import polars as pl
batting = pl.read_csv("data/lahman_1871-2024_csv/Batting.csv")
print(batting.head())

shape: (5, 24)
┌───────────┬────────┬───────┬────────┬───┬─────┬─────┬──────┬───────┐
│ playerID  ┆ yearID ┆ stint ┆ teamID ┆ … ┆ SH  ┆ SF  ┆ GIDP ┆ G_old │
│ ---       ┆ ---    ┆ ---   ┆ ---    ┆   ┆ --- ┆ --- ┆ ---  ┆ ---   │
│ str       ┆ i64    ┆ i64   ┆ str    ┆   ┆ i64 ┆ i64 ┆ i64  ┆ i64   │
╞═══════════╪════════╪═══════╪════════╪═══╪═════╪═════╪══════╪═══════╡
│ aardsda01 ┆ 2004   ┆ 1     ┆ SFN    ┆ … ┆ 0   ┆ 0   ┆ 0    ┆ null  │
│ aardsda01 ┆ 2006   ┆ 1     ┆ CHN    ┆ … ┆ 1   ┆ 0   ┆ 0    ┆ null  │
│ aardsda01 ┆ 2007   ┆ 1     ┆ CHA    ┆ … ┆ 0   ┆ 0   ┆ 0    ┆ null  │
│ aardsda01 ┆ 2008   ┆ 1     ┆ BOS    ┆ … ┆ 0   ┆ 0   ┆ 0    ┆ null  │
│ aardsda01 ┆ 2009   ┆ 1     ┆ SEA    ┆ … ┆ 0   ┆ 0   ┆ 0    ┆ null  │
└───────────┴────────┴───────┴────────┴───┴─────┴─────┴──────┴───────┘


In [16]:
batting_SLG = (batting
               .filter(pl.col("yearID") >= 1946)
               .group_by("yearID", "lgID")
               .agg(
                   total_bases = pl.sum("H") + pl.sum("2B") + (pl.sum("3B") * 2) + (pl.sum("HR")*3),
                   total_at_bats = pl.sum("AB"),
               )
               .with_columns(
                   league_wide_SLG = pl.col("total_bases") / pl.col("total_at_bats"),
                   DH_rule = pl.when((pl.col("lgID") == "AL") & (pl.col("yearID") >= 1973))
                       .then(pl.lit("DH"))
                       .when((pl.col("lgID") == "NL") & (pl.col("yearID") >= 2022))
                       .then(pl.lit("DH"))
                       .otherwise(pl.lit("No DH"))
               )
               .drop("total_bases", "total_at_bats")
              )

In [17]:
print(batting_SLG)

shape: (158, 4)
┌────────┬──────┬─────────────────┬─────────┐
│ yearID ┆ lgID ┆ league_wide_SLG ┆ DH_rule │
│ ---    ┆ ---  ┆ ---             ┆ ---     │
│ i64    ┆ str  ┆ f64             ┆ str     │
╞════════╪══════╪═════════════════╪═════════╡
│ 2021   ┆ NL   ┆ 0.406828        ┆ No DH   │
│ 1973   ┆ NL   ┆ 0.375853        ┆ No DH   │
│ 1988   ┆ AL   ┆ 0.390598        ┆ DH      │
│ 1981   ┆ NL   ┆ 0.36393         ┆ No DH   │
│ 2020   ┆ NL   ┆ 0.421171        ┆ No DH   │
│ …      ┆ …    ┆ …               ┆ …       │
│ 1992   ┆ AL   ┆ 0.385005        ┆ DH      │
│ 1982   ┆ NL   ┆ 0.372682        ┆ No DH   │
│ 1973   ┆ AL   ┆ 0.38145         ┆ DH      │
│ 1972   ┆ NL   ┆ 0.364599        ┆ No DH   │
│ 2005   ┆ NL   ┆ 0.414072        ┆ No DH   │
└────────┴──────┴─────────────────┴─────────┘


In [18]:
batting_SLG.write_csv("data/batting_SLG.csv")